***GENERATED CODE FOR dp1 PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
from pyspark.sql.functions import dayofmonth, month, year, col
import json
from pyspark.sql.functions import col, unix_timestamp
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def ConvertToDate(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]
    dateFormate = transformationData["dateFormat"]
    dfReturn = dfReturn.withColumn(feature, unix_timestamp(feature, dateFormate)
                                   .cast("double").cast("timestamp"))
    dfReturn = dfReturn.withColumn(
        feature + '_dayofmonth', dayofmonth(col(feature)))
    dfReturn = dfReturn.withColumn(feature + '_month', month(col(feature)))
    dfReturn = dfReturn.withColumn(feature + '_year', year(col(feature)))
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Loan_ID', 'transformation_label': 'String Indexer'}], 'feature': 'Loan_ID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '50', 'mean': '', 'stddev': '', 'min': 'xqd20160003', 'max': 'xqd88160031', 'missing': '0', 'distinct': '50'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Loan_ID'}, {'feature_label': 'Loan_ID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Loan_ID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'loan_status', 'transformation_label': 'String Indexer'}], 'feature': 'loan_status', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '50', 'mean': '', 'stddev': '', 'min': 'PAIDOFF', 'max': 'PAIDOFF', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'loan_status'}, {'feature_label': 'loan_status', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('loan_status')
        transformationDF = ConvertToDate(transformationDF, {'transformationsData': [{'feature_label': 'effective_date', 'dateFormat': 'dd/MM/yyyy', 'transformation_label': 'Convert to Date'}], 'feature': 'effective_date', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '50', 'mean': '', 'stddev': '', 'min': '09/08/2016', 'max': '09/11/2016', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'Convert to Date', 'selectedAsDefault': 1}], 'updatedLabel': 'effective_date'}, {'feature_label': 'effective_date', 'dateFormat': 'dd/MM/yyyy', 'transformation_label': 'Convert to Date'})
        transformationDF = transformationDF.drop('effective_date')
        transformationDF = ConvertToDate(transformationDF, {'transformationsData': [{'feature_label': 'due_date', 'dateFormat': 'MM-dd-yyyy', 'transformation_label': 'Convert to Date'}], 'feature': 'due_date', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '50', 'mean': '', 'stddev': '', 'min': '10/07/2016', 'max': '9/25/2016', 'missing': '0', 'distinct': '10'}, 'transformation': [{'transformation': 'Convert to Date', 'selectedAsDefault': 1}], 'updatedLabel': 'due_date'}, {'feature_label': 'due_date', 'dateFormat': 'MM-dd-yyyy', 'transformation_label': 'Convert to Date'})
        transformationDF = transformationDF.drop('due_date')
        transformationDF = ConvertToDate(transformationDF, {'transformationsData': [{'feature_label': 'paid_off_time', 'dateFormat': 'MM_dd_yyyy HH_mm_ss', 'transformation_label': 'Convert to Date'}], 'feature': 'paid_off_time', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '50', 'mean': '', 'stddev': '', 'min': '09/09/2016 13:45', 'max': '9/29/2016 12:18', 'missing': '0', 'distinct': '47'}, 'transformation': [{'transformation': 'Convert to Date', 'selectedAsDefault': 1}], 'updatedLabel': 'paid_off_time'}, {'feature_label': 'paid_off_time', 'dateFormat': 'MM_dd_yyyy HH_mm_ss', 'transformation_label': 'Convert to Date'})
        transformationDF = transformationDF.drop('paid_off_time')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'education', 'transformation_label': 'String Indexer'}], 'feature': 'education', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '50', 'mean': '', 'stddev': '', 'min': 'Bechalor', 'max': 'college', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'education'}, {'feature_label': 'education', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('education')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '50', 'mean': '', 'stddev': '', 'min': 'female', 'max': 'male', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***CONNECTOR FUNCTIONS TO WRITE DATA.***

In [ ]:
import datetime
import requests
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class NumtraConnector:

    def put(inStages, inStagesData, stageId, spark, config):
        path = eval(config)['server_url']
        baseType = eval(config)['baseType']
        results_url = eval(config)['results_url']
        server = eval(config)['server']
        originalfile = eval(config)['orignalKey']
        eval(config)['pathOnly']
        filename = eval(config)['filename']
        eval(config)['ser']
        eval(config)['user']
        eval(config)['password']
        eval(config)['authSource']
        eval(config)['user_id']
        eval(config)['parent_id']
        eval(config)['project_id']
        time = str(int(datetime.datetime.now().timestamp()))

        inStagesData[inStages[0]]

        print(path)
        print(baseType)
        print(results_url)
        print(server)
        print(originalfile)
        print(filename)

        args = {
            'url': path,
            'baseType': baseType,
            'originalfile': originalfile,
            'filename': time + filename
        }

        response = requests.post(results_url, args)
        return response


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://0.0.0.0:0000').getOrCreate()
#%run dp1Hooks.ipynb
try:
	#sourcePreExecutionHook()

	loanpaymenttest = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/Loan_payment_test.csv', 'filename': 'Loan_payment_test.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://10.0.5.54', 'port': '50070', 'dirPath': '/FileStore/platform', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/'}")
	#sourcePostExecutionHook(loanpaymenttest)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run dp1Hooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(loanpaymenttest,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "Loan_ID", "transformation_label": "String Indexer"}], "feature": "Loan_ID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "50", "mean": "", "stddev": "", "min": "xqd20160003", "max": "xqd88160031", "missing": "0", "distinct": "50"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Loan_ID"}, {"transformationsData": [{"feature_label": "loan_status", "transformation_label": "String Indexer"}], "feature": "loan_status", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "50", "mean": "", "stddev": "", "min": "PAIDOFF", "max": "PAIDOFF", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "loan_status"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Principal", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "50", "mean": "924.0", "stddev": "155.92", "min": "300", "max": "1000", "missing": "0"}, "updatedLabel": "Principal"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "terms", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "50", "mean": "21.84", "stddev": "8.92", "min": "7", "max": "30", "missing": "0"}, "updatedLabel": "terms"}, {"transformationsData": [{"feature_label": "effective_date", "dateFormat": "dd/MM/yyyy", "transformation_label": "Convert to Date"}], "feature": "effective_date", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "50", "mean": "", "stddev": "", "min": "09/08/2016", "max": "09/11/2016", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "Convert to Date", "selectedAsDefault": 1}], "updatedLabel": "effective_date"}, {"transformationsData": [{"feature_label": "due_date", "dateFormat": "MM-dd-yyyy", "transformation_label": "Convert to Date"}], "feature": "due_date", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "50", "mean": "", "stddev": "", "min": "10/07/2016", "max": "9/25/2016", "missing": "0", "distinct": "10"}, "transformation": [{"transformation": "Convert to Date", "selectedAsDefault": 1}], "updatedLabel": "due_date"}, {"transformationsData": [{"feature_label": "paid_off_time", "dateFormat": "MM_dd_yyyy HH_mm_ss", "transformation_label": "Convert to Date"}], "feature": "paid_off_time", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "50", "mean": "", "stddev": "", "min": "09/09/2016 13:45", "max": "9/29/2016 12:18", "missing": "0", "distinct": "47"}, "transformation": [{"transformation": "Convert to Date", "selectedAsDefault": 1}], "updatedLabel": "paid_off_time"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "past_due_days", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "50", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "past_due_days"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "age", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "50", "mean": "31.42", "stddev": "6.45", "min": "21", "max": "50", "missing": "0"}, "updatedLabel": "age"}, {"transformationsData": [{"feature_label": "education", "transformation_label": "String Indexer"}], "feature": "education", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "50", "mean": "", "stddev": "", "min": "Bechalor", "max": "college", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "education"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "50", "mean": "", "stddev": "", "min": "female", "max": "male", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***WRITING DATAFRAME***

In [ ]:
#%run dp1Hooks.ipynb
try:
	#sinkPreExecutionHook()

	datapipeline = NumtraConnector.fetch(spark, "{'samplefile': '/FileStore/platform/sampleData/63db8baa9f5db4fdf5b0b884/part-00000-e14a4867-efcc-47f8-a8b2-718cc1df46bc-c000.csv', 'samplecount': 500, 'originalcount': 500, 'orignalKey': '/FileStore/platform/uploadedSourceFiles/Loan_payment_test.csv', 'pathOnly': '/dataPipeline', 'project_id': '63c0fe048b96720de22fd1f9', 'parent_id': '63c0fe048b96720de22fd1f9', 'original_schema': [{'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '63db8d4f9f5db4fdf5b0b904', 'field': 'Principal', 'alias': 'Principal', 'type': 'numeric', 'position': '0'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '63db8d4f9f5db4fdf5b0b905', 'field': 'terms', 'alias': 'terms', 'type': 'numeric', 'position': '1'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '63db8d4f9f5db4fdf5b0b906', 'field': 'past_due_days', 'alias': 'past_due_days', 'type': 'numeric', 'position': '2'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '63db8d4f9f5db4fdf5b0b907', 'field': 'age', 'alias': 'age', 'type': 'numeric', 'position': '3'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '63db8d4f9f5db4fdf5b0b908', 'field': 'Loan_ID_stringindexer', 'alias': 'Loan_ID_stringindexer', 'type': 'real', 'position': '4'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '63db8d4f9f5db4fdf5b0b909', 'field': 'loan_status_stringindexer', 'alias': 'loan_status_stringindexer', 'type': 'numeric', 'position': '5'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '63db8d4f9f5db4fdf5b0b90a', 'field': 'education_stringindexer', 'alias': 'education_stringindexer', 'type': 'numeric', 'position': '6'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '63db8d4f9f5db4fdf5b0b90b', 'field': 'Gender_stringindexer', 'alias': 'Gender_stringindexer', 'type': 'numeric', 'position': '7'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '63db8d4f9f5db4fdf5b0b90c', 'field': 'effective_date_converttodate', 'alias': 'effective_date_converttodate', 'type': 'date', 'position': '8'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '63db8d4f9f5db4fdf5b0b90d', 'field': 'due_date_converttodate', 'alias': 'due_date_converttodate', 'type': 'date', 'position': '9'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '63db8d4f9f5db4fdf5b0b90e', 'field': 'paid_off_time_converttodate', 'alias': 'paid_off_time_converttodate', 'type': 'date', 'position': '10'}], 'actual_schema': [{'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '63db8d4e9f5db4fdf5b0b8f9', 'field': 'Loan_ID', 'alias': 'Loan_ID', 'type': 'String', 'position': '0'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '63db8d4e9f5db4fdf5b0b8fa', 'field': 'loan_status', 'alias': 'loan_status', 'type': 'String', 'position': '1'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '63db8d4e9f5db4fdf5b0b8fb', 'field': 'Principal', 'alias': 'Principal', 'type': 'real', 'position': '2'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '63db8d4e9f5db4fdf5b0b8fc', 'field': 'terms', 'alias': 'terms', 'type': 'real', 'position': '3'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '63db8d4f9f5db4fdf5b0b8fd', 'field': 'effective_date', 'alias': 'effective_date', 'type': 'date', 'position': '4'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '63db8d4f9f5db4fdf5b0b8fe', 'field': 'due_date', 'alias': 'due_date', 'type': 'date', 'position': '5'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '63db8d4f9f5db4fdf5b0b8ff', 'field': 'paid_off_time', 'alias': 'paid_off_time', 'type': 'date', 'position': '6'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '63db8d4f9f5db4fdf5b0b900', 'field': 'past_due_days', 'alias': 'past_due_days', 'type': 'real', 'position': '7'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '63db8d4f9f5db4fdf5b0b901', 'field': 'age', 'alias': 'age', 'type': 'real', 'position': '8'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '63db8d4f9f5db4fdf5b0b902', 'field': 'education', 'alias': 'education', 'type': 'String', 'position': '9'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '63db8d4f9f5db4fdf5b0b903', 'field': 'Gender', 'alias': 'Gender', 'type': 'String', 'position': '10'}], 'server': 'https://dev.numtra.com:443', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/', 'delimiter': ',', 'file_type': 'Delimeted', 'filename': 'AutoFEData.csv', 'token': '', 'domain': '', 'is_header': 'Use Header Line', 'url': '/FileStore/platform/uploadedSourceFiles/part-00000-b4f0430c-4233-4cb9-a143-4c8a34985393-c000.csv', 'results_url': 'http://dev.numtra.com:4040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
